## Projeto de Redes Sociais

In [56]:
import freeman as fm
import seaborn as sns
import statsmodels.api as sm
import networkx as nx
import pylab
from collections import defaultdict



In [59]:
g = fm.load('data/lol_teams.gml')

In [61]:
g.set_all_nodes(size=15, labpos='hover')

# Deixar as arestas semi-transparentes para
# a visualização ficar um pouco mais leve.
# g.set_all_edges(color=(0, 0, 0, 0.25))

# Usar um algoritmo especial para posicionar
# os nós de acordo com grupos.
g.move('kamada_kawai')

g.draw()

In [ ]:
schools = defaultdict(list)


# for n in g.nodes:
#     schools['id'].append(n)

for n in g.nodes:
    schools['isatm'].append(float(g.nodes[n]['isatm']))

for n in g.nodes:
    schools['closeness'].append(nx.closeness_centrality(g, n))

print(len(schools['isatm']))

In [ ]:
ax = pylab.subplot(111)
ax.scatter(school_closeness, school_scores)
pylab.xlabel('Closeness')
pylab.ylabel('Performance')
pylab.suptitle('Closeness x Performance', fontsize=16)
ax.figure.show()

In [ ]:
model = sm.OLS(school_scores, school_closeness)
result = model.fit()
result.summary()